In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [4]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [5]:
df.isnull().sum()

age                 0
affordibility       0
bought_insurance    0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train,x_test,Y_train,y_test = train_test_split(df.drop('bought_insurance',axis='columns'),df.bought_insurance,test_size=0.25)

In [7]:
len(X_train)

21

In [ ]:
len(x_test)

In [9]:
X_train_scaled = X_train
X_train_scaled['age'] = X_train_scaled['age']/100

x_test_scaled = x_test 
x_test_scaled['age'] = x_test_scaled['age']/100

In [10]:
model = keras.Sequential(
   [ keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zero')]
)

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
model.fit(X_train_scaled,Y_train,epochs=5000)

Streaming output truncated to the last 5000 lines.
Epoch 2289/5000
1/1 [==============================] - 0s 20ms/step - loss: 0.5317 - accuracy: 0.9048
Epoch 2290/5000
1/1 [==============================] - 0s 28ms/step - loss: 0.5316 - accuracy: 0.9048
Epoch 2291/5000
1/1 [==============================] - 0s 14ms/step - loss: 0.5316 - accuracy: 0.9048
Epoch 2292/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.5315 - accuracy: 0.9048
Epoch 2293/5000
1/1 [==============================] - 0s 17ms/step - loss: 0.5315 - accuracy: 0.9048
Epoch 2294/5000
1/1 [==============================] - 0s 22ms/step - loss: 0.5314 - accuracy: 0.9048
Epoch 2295/5000
1/1 [==============================] - 0s 30ms/step - loss: 0.5314 - accuracy: 0.9048
Epoch 2296/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5313 - accuracy: 0.9048
Epoch 2297/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.5313 - accuracy: 0.9048
Epoch 2298/5000
1/1 [==========

In [12]:
model.evaluate(x_test,y_test)

1/1 [==============================] - 0s 229ms/step - loss: 0.5574 - accuracy: 0.7143


[0.5573797821998596, 0.7142857313156128]

In [13]:
model.predict(x_test_scaled)

1/1 [==============================] - 0s 107ms/step


array([[0.6299038 ],
       [0.29269287],
       [0.7273042 ],
       [0.64049345],
       [0.6782957 ],
       [0.6682428 ],
       [0.8069281 ]], dtype=float32)

In [14]:
y_test

4     1
21    0
25    1
3     0
6     0
14    1
8     1
Name: bought_insurance, dtype: int64

In [15]:
coef,intercept = model.get_weights()
coef

array([[5.6147985],
       [0.2911855]], dtype=float32)

In [16]:
import math

In [47]:
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [48]:
sigmoid(23)

0.9999999998973812

In [49]:
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [50]:
def prediction_function(age,affordibility):
  weighted_sum = coef[0]*age+coef[1]*affordibility+intercept
  return sigmoid(weighted_sum)

In [51]:
prediction_function(.22,1)

0.30666706724219867

In [52]:
prediction_function(.18,1)

0.26108447528106615

In [53]:
prediction_function(.47,1)

0.6428951330003795

In [54]:
def sigmoid_numpy(x):
  return 1/(1+np.exp(-x))

In [57]:
sigmoid_numpy(np.array([12,3,1]))

array([0.99999386, 0.95257413, 0.73105858])

In [84]:
def log_loss(y,y_pred):
  epsilon = 1e-10
  y_new = [max(i,epsilon) for i in y_pred]
  y_new = [min(i,1-epsilon) for i in y_new]
  y_new = np.array(y_new)
  loss = -np.mean(y*np.log(y_new)+(1-y)*np.log(1-y_new))
  return loss

In [94]:
def gredient_decent(age,affordibility,y_true,epochs,loss_thresold):
  w1=w2=1
  bias=0
  learning_rate=0.01
  n=len(age)
  for i in range(epochs):
    weigth_sum=w1*age+w2*affordibility+bias
    y_predicted = sigmoid_numpy(weigth_sum)
    loss = log_loss(y_true,y_predicted)
    w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
    w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_true))

    bias_d = np.mean(y_predicted-y_true)

    w1 = w1+learning_rate*w1d
    w2 = w2+learning_rate*w2d
    bias = bias+learning_rate*bias_d

    print(f'Epoch:{i} , w1:{w1},w2:{w2},bias:{bias},loss:{loss}' )

    if loss<=loss_thresold:
        break
      
  return w1,w2,bias

In [96]:
gredient_decent(X_train_scaled['age'],X_train_scaled['affordibility'],Y_train,1000,0.4328)

Epoch:0 , w1:1.000419484862143,w2:1.001903217288318,bias:0.0025967677473833824,loss:0.7826781957626314
Epoch:1 , w1:1.0008415658536112,w2:1.0038117353071625,bias:0.0052007289485038214,loss:0.7837338298476043
Epoch:2 , w1:1.001266244419001,w2:1.0057255545791848,bias:0.0078118888732077955,loss:0.7847954491332086
Epoch:3 , w1:1.0016935219547072,w2:1.0076446755106003,bias:0.01043025266527065,loss:0.7858630739890176
Epoch:4 , w1:1.0021233998087287,w2:1.0095690983908259,bias:0.013055825341805851,loss:0.7869367246960073
Epoch:5 , w1:1.0025558792804783,w2:1.0114988233921256,bias:0.015688611792683233,loss:0.7880164214447879
Epoch:6 , w1:1.0029909616205972,w2:1.0134338505692688,bias:0.018328616779956362,loss:0.7891021843338338
Epoch:7 , w1:1.0034286480307713,w2:1.0153741798591962,bias:0.020975844937299145,loss:0.7901940333677169
Epoch:8 , w1:1.0038689396635534,w2:1.017319811080697,bias:0.0236303007694518,loss:0.7912919884553392
Epoch:9 , w1:1.0043118376221878,w2:1.0192707439340956,bias:0.0262919

(2.1096856428758484, 4.079092239166188, 4.606049943751523)